In [1]:
import requests
import json
import pandas as pd
import numpy as np
import ift6758
from pathlib import Path

In [2]:
import os
from pathlib import Path
import logging
from flask import Flask, jsonify, request, abort, g
import xgboost
import requests
import pandas as pd
from comet_ml import API
import joblib

import sklearn
import ift6758

In [3]:
try:
    hey = requests.get("http://0.0.0.0:5000/hello")
except Exception as e:
    print(e)

In [4]:
hey.text

'Hello!'

In [5]:
logs = requests.get("http://0.0.0.0:5000/logs")
logs.json()[-5:]

['2022-12-18 23:24:40,101;INFO;Prediction end',
 '2022-12-19 04:30:08,078;INFO;Before first request - Start',
 '2022-12-19 04:30:08,675;INFO;Before first request - End',
 '2022-12-19 04:30:08,677;INFO;Accessed page /hello - Hello!',
 '2022-12-19 04:30:08,701;INFO;Accessed page /logs']

In [6]:
default_model = {
    'workspace': 'ift-6758-projet-quipe-13',
    'model': 'xgboost-best-all-features',
    'version': '1.0.0'
}

In [7]:
model = requests.post("http://0.0.0.0:5000/download_registry_model", json=default_model)

In [8]:
model.text

'"Updated from local folder."\n'

In [9]:
model

<Response [200]>

In [10]:
df = pd.DataFrame(np.random.randn(5266, 29))

In [11]:
X_dict = {}
X_values = df.values.tolist()
X_dict['values'] = X_values

In [12]:
list(X_dict.keys())[0]

'values'

In [13]:
pred = requests.post("http://0.0.0.0:5000/predict", json=X_dict)

In [14]:
pred.json()[:10]

[0.14881837368011475,
 0.36324939131736755,
 0.1754634976387024,
 0.19361110031604767,
 0.342043399810791,
 0.13273927569389343,
 0.45606961846351624,
 0.2758273184299469,
 0.39434486627578735,
 0.40354111790657043]

In [15]:
df = pd.DataFrame(np.random.randn(5266, 27))

In [16]:
X_dict = {}
X_values = df.values.tolist()
X_dict['values'] = X_values

In [17]:
pred = requests.post("http://0.0.0.0:5000/predict", json=X_dict)

In [18]:
pred.json()[:10]

[0.2581057846546173,
 0.2821357250213623,
 0.15087231993675232,
 0.13990314304828644,
 0.31382086873054504,
 0.19145913422107697,
 0.2737134099006653,
 0.16317468881607056,
 0.28011423349380493,
 0.5015616416931152]

In [19]:
df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.314441,-0.128778,-0.770124,-1.348072,0.607685,-0.210639,-0.544075,-0.365823,-0.369326,-0.168643,...,-0.314148,-0.135038,0.954199,-1.141506,1.018330,-0.758034,-0.251249,-0.260766,0.989570,-0.538327
1,-1.555199,0.479039,-0.496223,0.432008,0.156626,0.005134,0.945148,-0.556190,0.236113,1.222855,...,1.597688,-0.491842,1.472823,-0.473942,0.187930,0.218855,0.695292,-0.752812,-0.011255,1.222044
2,0.674996,-0.354226,0.529731,-0.311066,1.087795,-0.596102,-1.133463,-0.087565,0.791539,0.265699,...,1.353883,0.322616,-0.779567,-0.324279,-1.252193,0.533928,-0.393589,-0.152217,-0.607616,-0.064996
3,-0.971353,-0.529260,-0.612571,0.910791,-0.097262,-2.016272,-0.062618,1.441894,-1.013383,0.105240,...,0.270339,1.661559,-1.431808,-1.743928,0.098326,-0.039271,0.513137,0.495054,0.457317,-0.369797
4,-0.525574,-0.492881,-0.618570,1.778219,0.909482,-0.868474,-0.753441,0.005551,-1.497077,1.009799,...,-0.302135,-0.770287,-0.059553,-0.605963,-0.674328,-0.998330,0.141025,-1.570002,0.195214,0.539553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5261,-0.284139,-1.651831,-1.054342,0.572656,-1.378107,0.821235,-0.070877,0.001593,0.922347,-1.964685,...,-0.254746,-0.532356,-1.234900,0.995977,1.059190,-1.073477,2.049936,0.891027,2.397933,-1.294907
5262,-0.643911,0.612951,-0.194824,0.767918,0.726322,-1.225412,-0.060520,-1.396778,1.653956,-0.668348,...,-0.152009,-1.093858,1.300815,0.424382,0.990389,-1.152223,-0.186793,0.133913,-0.068834,0.064354
5263,-0.282999,-0.607342,0.933987,1.339848,-0.627867,0.767836,-0.062189,-1.479858,-1.722287,-1.062303,...,-1.404846,-0.677992,0.660994,0.426980,-1.550055,-1.076564,-0.348365,1.193535,-1.158135,0.421281
5264,-0.800487,0.643641,-0.574944,0.121939,1.810656,0.252993,-1.486773,-0.081766,0.807490,0.658046,...,-1.121907,0.913671,-1.620122,-1.281266,1.251629,-0.321120,-1.272483,0.491101,-0.370764,-0.656110


In [27]:
a = np.array([np.nan])

In [28]:
xd = pd.DataFrame(a)

In [29]:
xd

,0
0,NaN


In [38]:
if xd.values is None:
    print('hello')

In [41]:
xd.values == npnp.nan

array([[False]])

In [22]:
Y = None

In [24]:
if Y is None:
    print('hi')

hi


In [26]:
features = [
                'periodTimeSec', 'period', 'coordinate_x', 'coordinate_y',
                'distanceFromGoal', 'shotAngle', 'lastEventCoord_x', 'lastEventCoord_y',
                'timeDifference', 'distanceDifference', 'rebound',
                'shotAngleDifference', 'speed', 'Backhand', 'Deflected', 'Slap Shot',
                'Snap Shot', 'Tip-In', 'Wrap-around', 'Wrist Shot', 'BLOCKED_SHOT',
                'FACEOFF', 'GIVEAWAY', 'GOAL', 'HIT', 'MISSED_SHOT', 'SHOT',
                'TAKEAWAY', 'PENALTY'
            ]

In [27]:
xxdd = pd.DataFrame(np.nan, index=[0], columns=features)

In [28]:
xxdd

,periodTimeSec,period,coordinate_x,coordinate_y,distanceFromGoal,shotAngle,lastEventCoord_x,lastEventCoord_y,timeDifference,distanceDifference,...,Wrist Shot,BLOCKED_SHOT,FACEOFF,GIVEAWAY,GOAL,HIT,MISSED_SHOT,SHOT,TAKEAWAY,PENALTY
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
